In [32]:
import re
import os
import sys
import pandas as pd
import statsmodels.formula.api as smf
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import json
from scipy import stats
import csv
import pprint
from collections import defaultdict
import time
import datetime
import math
# import matplotlib.pyplot as plt; plt.rcdefaults()
# from IPython.display import Markdown, display

In [7]:
TIME_CHUNK_IN_DAYS = 10
QUARANTINE_DATE = "28/09/2018"
quarDataUnix = time.mktime(datetime.datetime.strptime(QUARANTINE_DATE, "%d/%m/%Y").timetuple())

CONTROL_SUBREDDITS = "/mnt/storage/quarantine/data/top-100-controls-subreddits.csv"
COMMENTS_FOLDER = "/mnt/storage/quarantine/data/control_subreddit_comments/"
SUBMISSIONS_FOLDER = "/mnt/storage/quarantine/data/control_subreddit_submissions/"
ITS_RESULTS_CONTROL_SUBS = "/mnt/storage/quarantine/data/its-results-control-subreddits.csv"

pd.options.display.max_rows = 1999

In [10]:
###load files and preprocess dataframes
def getTimeChunkIndex(timeStamp):
    timeStamp = float(timeStamp)
    timeDiff = timeStamp - quarDataUnix
    timeDiffDays = float(timeDiff)/(24*60*60) 
    chunkIndex = math.floor(timeDiffDays/TIME_CHUNK_IN_DAYS)
    return int(chunkIndex)

def load_data(subreddit):
    comments_data = pd.read_csv(COMMENTS_FOLDER + subreddit + ".csv")
    submissions_data = pd.read_csv(SUBMISSIONS_FOLDER + subreddit + ".csv")
    return comments_data, submissions_data

def preprocess_data(comments_data, submissions_data):
    ###preprocessing - retain only columns of importance
    comments_data = comments_data[['author', 'created_utc', 'body']]
    submissions_data = submissions_data[['author', 'created_utc', 'title', 'selftext']]
    
    ###make the column names the same
    submissions_data.rename(columns={'title': 'body'}, inplace=True)
    comments_data['selftext'] = 'EMPTY'
    comments_data['type'] = 'comment'
    submissions_data['type'] = 'submission'
    
    ###merge comments + submissions; remove rows with null author or created_utc
    subreddit_timeline = pd.concat([comments_data, submissions_data], ignore_index=True, sort=True)
    subreddit_timeline = subreddit_timeline.dropna(subset=['author', 'created_utc'])
    
    ###typecasting to prevent errors
    subreddit_timeline = subreddit_timeline[pd.to_numeric(subreddit_timeline['created_utc'], errors='coerce').notnull()]
    subreddit_timeline['author']=subreddit_timeline.author.astype('str')
    subreddit_timeline['body']=subreddit_timeline.body.astype('str')
    subreddit_timeline['selftext']=subreddit_timeline.selftext.astype('str')
    subreddit_timeline['created_utc']=subreddit_timeline.created_utc.astype('float')
    subreddit_timeline['type']=subreddit_timeline.type.astype('str')
    
    ###compute time variables for ITS
    subreddit_timeline['post_treatment'] = subreddit_timeline['created_utc'] > quarDataUnix
    subreddit_timeline['time'] = subreddit_timeline['created_utc'].apply(getTimeChunkIndex)
    
    return subreddit_timeline

def compute_posting_and_removal_volume(data):
    TrpRemovalCount = {}
    TotalNumberComments = {}
    authorsInEachChunk = defaultdict(set)
    for i, row in data.iterrows():
            body = row['body']
            selftext = row['selftext']
            postTime = row['created_utc']
            timeChunkIndex = getTimeChunkIndex(postTime)

            if timeChunkIndex in TotalNumberComments:
                TotalNumberComments[timeChunkIndex] += 1
            else:
                TotalNumberComments[timeChunkIndex] = 1

            ###count #removed
            if ((body == "[removed]") | (selftext == "[removed]")):
                if timeChunkIndex in TrpRemovalCount:
                    TrpRemovalCount[timeChunkIndex] += 1
                else:
                    TrpRemovalCount[timeChunkIndex] = 1
                    
            author = row['author']
            if (author != "[deleted]"):
                authorsInEachChunk[timeChunkIndex].add(author)                                                

    ##get the removal rate: i.e., #removed comments normalized by #total comments
    TrpRemovalRate = {}
    for k in TrpRemovalCount:
        if TotalNumberComments[k] == 0:
            TrpRemovalRate[k] = 0
        else:      
            TrpRemovalRate[k] = float(TrpRemovalCount[k])/(TotalNumberComments[k])            
            
    # Get influx of new users
    lists = sorted(authorsInEachChunk.items())
    seenUsersSet = set()
    for ulist in lists[0:3]:
        chunkUsers = ulist[1]
        seenUsersSet = seenUsersSet.union(chunkUsers)

    newUsersCount = defaultdict(int)
    for chunkIndex in TotalNumberComments.keys():
        newUsersCount[chunkIndex] = 1
    for ulist in lists[4:]:
        chunkIndex = ulist[0]
        chunkUsers = ulist[1]
        newUsersSet = chunkUsers.difference(seenUsersSet)
        newUsersCount[chunkIndex] += len(newUsersSet)
        seenUsersSet = seenUsersSet.union(chunkUsers)                

    return TotalNumberComments, TrpRemovalRate, newUsersCount
    
def get_effect(coeff):
    return np.sign(coeff) * (100* (1.0 - np.exp(coeff)))

def its_posting_volume(TotalNumberComments, subreddit): 
    # posting volume
    TotalNumberComments_df = pd.DataFrame()
    TotalNumberComments_df['time'] = TotalNumberComments.keys()
    TotalNumberComments_df['counts'] = TotalNumberComments.values()
    TotalNumberComments_df['post_treatment'] = TotalNumberComments_df['time'] >= 0

    # ITS
    model = smf.poisson('counts ~ time + post_treatment', data = TotalNumberComments_df)
    results = model.fit()
    
    coeff = results.params[1]
    pval = results.pvalues[1]
    effect = get_effect(coeff)
    return coeff, pval, effect

def its_removal_rate(TrpRemovalRate, subreddit):
    TrpRemovalRate_df = pd.DataFrame()
    TrpRemovalRate_df['time'] = TrpRemovalRate.keys()
    TrpRemovalRate_df['counts'] = TrpRemovalRate.values()
    TrpRemovalRate_df['post_treatment'] = TrpRemovalRate_df['time'] >= 0

    # ITS
    model = smf.ols('counts ~ time + post_treatment', data = TrpRemovalRate_df)
    results = model.fit()
    
    coeff = results.params[1]
    pval = results.pvalues[1]
    effect = np.sign(coeff) * coeff * 100.0
    return coeff, pval, effect

def its_poisson(myDict, subreddit): 
    # posting volume
    df = pd.DataFrame()
    df['time'] = myDict.keys()
    df['counts'] = myDict.values()
    df['post_treatment'] = df['time'] >= 0

    # ITS
    model = smf.poisson('counts ~ time + post_treatment', data = df)
    results = model.fit()
    
    coeff = results.params[1]
    pval = results.pvalues[1]
    effect = get_effect(coeff)
    return coeff, pval, effect

In [7]:
control_subs_list = pd.read_csv(CONTROL_SUBREDDITS, names = ['subreddit']).subreddit.tolist()

post_volume_subreddit_list = []
post_volume_coeff_list = []
post_volume_pval_list = []
post_volume_effect_list = []

removal_rate_subreddit_list = []
removal_rate_coeff_list = []
removal_rate_pval_list = []
removal_rate_effect_list = []

new_users_subreddit_list = []
new_users_coeff_list = []
new_users_pval_list = []
new_users_effect_list = []
count = 0

for subreddit in control_subs_list:
    print(count, subreddit)
    comments_data, submissions_data = load_data(subreddit)
    subreddit_timeline = preprocess_data(comments_data, submissions_data)
    TotalNumberComments, TrpRemovalRate, newUsersCount = compute_posting_and_removal_volume(subreddit_timeline)
    
    coeff, pval, effect = its_posting_volume(TotalNumberComments, subreddit)
    post_volume_subreddit_list.append(subreddit)
    post_volume_coeff_list.append(coeff)
    post_volume_pval_list.append(pval)
    post_volume_effect_list.append(effect)
    
    coeff, pval, effect = its_removal_rate(TrpRemovalRate, subreddit)
    removal_rate_subreddit_list.append(subreddit)
    removal_rate_coeff_list.append(coeff)
    removal_rate_pval_list.append(pval)
    removal_rate_effect_list.append(effect)
    
    coeff, pval, effect = its_poisson(newUsersCount, subreddit)
    new_users_subreddit_list.append(subreddit)
    new_users_coeff_list.append(coeff)
    new_users_pval_list.append(pval)
    new_users_effect_list.append(effect)
    
    count += 1

0 asktrp


/nethome/sjhaver3/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2901: DtypeWarning: Columns (29) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/nethome/sjhaver3/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)
/nethome/sjhaver3/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/nethome/sjhaver3/anaconda3/lib/python3.7/site-packages/ipykernel_la

Optimization terminated successfully.
         Current function value: 55.170273
         Iterations 4
Optimization terminated successfully.
         Current function value: 61.446268
         Iterations 4
1 askMRP
Optimization terminated successfully.
         Current function value: 49.520569
         Iterations 5
Optimization terminated successfully.
         Current function value: 9.064661
         Iterations 4
2 The30DayChallenge
Optimization terminated successfully.
         Current function value: 7.857486
         Iterations 7
Optimization terminated successfully.
         Current function value: 2.739024
         Iterations 6
3 marriedredpill


/nethome/sjhaver3/anaconda3/lib/python3.7/site-packages/statsmodels/regression/linear_model.py:1510: RuntimeWarning: divide by zero encountered in double_scalars
  return np.dot(wresid, wresid) / self.df_resid
/nethome/sjhaver3/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:1181: RuntimeWarning: invalid value encountered in multiply
  cov_p = self.normalized_cov_params * scale
/nethome/sjhaver3/anaconda3/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/nethome/sjhaver3/anaconda3/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
/nethome/sjhaver3/anaconda3/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:1821: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


Optimization terminated successfully.
         Current function value: 24.582111
         Iterations 4
Optimization terminated successfully.
         Current function value: 9.104024
         Iterations 4
4 RPChristians
Optimization terminated successfully.
         Current function value: 20.880311
         Iterations 4
Optimization terminated successfully.
         Current function value: 4.089850
         Iterations 4
5 PurplePillDebate
Optimization terminated successfully.
         Current function value: 326.207864
         Iterations 4
Optimization terminated successfully.
         Current function value: 23.640106
         Iterations 5
6 WhereAreAllTheGoodMen
Optimization terminated successfully.
         Current function value: 49.252295
         Iterations 5
Optimization terminated successfully.
         Current function value: 18.278321
         Iterations 5
7 RedPillWomen
Optimization terminated successfully.
         Current function value: 29.340813
         Iterations 4
O

...

Optimization terminated successfully.
         Current function value: 41.315086
         Iterations 4
Optimization terminated successfully.
         Current function value: 42.986333
         Iterations 5
57 tressless
Optimization terminated successfully.
         Current function value: 49.144278
         Iterations 4
Optimization terminated successfully.
         Current function value: 34.098308
         Iterations 4
58 AskThe_Donald
Optimization terminated successfully.
         Current function value: 58.514538
         Iterations 4
Optimization terminated successfully.
         Current function value: 35.543650
         Iterations 4
59 Supplements
Optimization terminated successfully.
         Current function value: 19.794739
         Iterations 5
Optimization terminated successfully.
         Current function value: 29.544915
         Iterations 5
60 EqualAttraction
Optimization terminated successfully.
         Current function value: 27.160292
         Iterations 5
Optimizat

/nethome/sjhaver3/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2901: DtypeWarning: Columns (24,32) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


Optimization terminated successfully.
         Current function value: 37.704481
         Iterations 5
Optimization terminated successfully.
         Current function value: 57.799324
         Iterations 5
87 sales
Optimization terminated successfully.
         Current function value: 31.939109
         Iterations 4
Optimization terminated successfully.
         Current function value: 38.459644
         Iterations 5
88 lostgeneration
Optimization terminated successfully.
         Current function value: 54.216859
         Iterations 4
Optimization terminated successfully.
         Current function value: 37.803100
         Iterations 5
89 aznidentity
Optimization terminated successfully.
         Current function value: 37.935695
         Iterations 4
Optimization terminated successfully.
         Current function value: 21.774233
         Iterations 5
90 samharris
Optimization terminated successfully.
         Current function value: 205.109354
         Iterations 4
Optimization term

/nethome/sjhaver3/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2901: DtypeWarning: Columns (0,1,2,4,5,8,9,10,15,16,18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


Optimization terminated successfully.
         Current function value: 1618.671832
         Iterations 4
Optimization terminated successfully.
         Current function value: 152.561586
         Iterations 4
93 survivinginfidelity
Optimization terminated successfully.
         Current function value: 20.750518
         Iterations 4
Optimization terminated successfully.
         Current function value: 23.152141
         Iterations 5
94 selfimprovement
Optimization terminated successfully.
         Current function value: 22.317344
         Iterations 4
Optimization terminated successfully.
         Current function value: 51.348472
         Iterations 5
95 awakened
Optimization terminated successfully.
         Current function value: 26.815369
         Iterations 4
Optimization terminated successfully.
         Current function value: 18.796948
         Iterations 5
96 fragrance
Optimization terminated successfully.
         Current function value: 29.893321
         Iterations 4
Opt

In [8]:
post_volume_results_df = pd.DataFrame()
post_volume_results_df['subreddit'] = post_volume_subreddit_list
post_volume_results_df['pv-coeff'] = post_volume_coeff_list
post_volume_results_df['pv-pval'] = post_volume_pval_list
post_volume_results_df['pv-effect'] = post_volume_effect_list

removal_rate_results_df = pd.DataFrame()
removal_rate_results_df['subreddit'] = removal_rate_subreddit_list
removal_rate_results_df['rr-coeff'] = removal_rate_coeff_list
removal_rate_results_df['rr-pval'] = removal_rate_pval_list
removal_rate_results_df['rr-effect'] = removal_rate_effect_list

new_users_results_df = pd.DataFrame()
new_users_results_df['nu-coeff'] = new_users_coeff_list
new_users_results_df['nu-pval'] = new_users_pval_list
new_users_results_df['nu-effect'] = new_users_effect_list

results = pd.concat([post_volume_results_df, removal_rate_results_df.drop(['subreddit'], axis = 1), new_users_results_df], axis=1)

In [9]:
results.to_csv(ITS_RESULTS_CONTROL_SUBS, index = False)

In [10]:
results.head()

,subreddit,pv-coeff,pv-pval,pv-effect,rr-coeff,rr-pval,rr-effect,nu-coeff,nu-pval,nu-effect
0,asktrp,-0.062170,1.629548e-28,-6.027703,0.007179,0.017486,0.717903,-0.299875,1.065095e-21,-25.908923
1,askMRP,-0.064078,1.569385e-03,-6.206846,0.005066,0.513867,0.506620,-0.323684,2.441931e-03,-27.652107
2,The30DayChallenge,0.415308,1.309230e-01,-51.483724,0.000000,NaN,0.000000,-2.567027,3.141581e-03,-92.323661
3,marriedredpill,-0.134869,5.261136e-09,-12.616968,0.003107,0.608724,0.310688,-0.280435,3.514391e-03,-24.454471
4,RPChristians,-0.050485,1.649894e-01,-4.923188,-0.003850,0.572287,0.384976,-0.252503,1.378125e-01,-22.314595


In [33]:
results = pd.read_csv(ITS_RESULTS_CONTROL_SUBS)
results = results.dropna()

In [34]:
results

,subreddit,pv-coeff,pv-pval,pv-effect,rr-coeff,rr-pval,rr-effect,nu-coeff,nu-pval,nu-effect
0,asktrp,-0.062170,1.629548e-28,-6.027703,0.007179,0.017486,0.717903,-0.299875,1.065095e-21,-25.908923
1,askMRP,-0.064078,1.569385e-03,-6.206846,0.005066,0.513867,0.506620,-0.323684,2.441931e-03,-27.652107
3,marriedredpill,-0.134869,5.261136e-09,-12.616968,0.003107,0.608724,0.310688,-0.280435,3.514391e-03,-24.454471
4,RPChristians,-0.050485,1.649894e-01,-4.923188,-0.003850,0.572287,0.384976,-0.252503,1.378125e-01,-22.314595
5,PurplePillDebate,0.362312,0.000000e+00,-43.664722,-0.002470,0.402575,0.246998,0.348565,5.226601e-11,-41.703288
6,WhereAreAllTheGoodMen,0.125069,3.458103e-30,-13.322613,-0.000679,0.864972,0.067885,-0.134381,7.981577e-03,-12.574318
7,RedPillWomen,-0.116114,2.335605e-12,-10.962641,-0.015507,0.205099,1.550683,-0.433084,8.713819e-11,-35.149389
8,DarkEnlightenment,-0.017237,6.792202e-01,-1.708955,-0.007619,0.631752,0.761878,-0.455536,3.003887e-04,-36.589179
9,BattleOfTheSexes,0.998123,7.644497e-48,-171.318361,-0.008149,0.756503,0.814923,-0.804780,2.358060e-03,-55.281362
10,MGTOW2,-0.037297,1.811114e-01,-3.660974,0.009197,0.534904,0.919717,-0.554358,1.433404e-08,-42.555919


# BOOTSTRAPPING ANALYSIS:
How extreme is the effect on TRP compared to the 100 control subreddits?

### 1) Posting volumes

In [36]:
from random import choices

treatment_coeff = -0.7418

control_coeffs = results['pv-coeff'].tolist()
# control_coeffs = post_volume_coeff_list

sample_size = 50
iteration = 0
limit = 10000

pval_list = []

while(iteration < limit):
    iteration_coeffs = choices(control_coeffs, k=sample_size)
    pval = 0
    for coeff in iteration_coeffs:
        if treatment_coeff < coeff:
            pval += 1
    pval_list.append(pval)
    iteration += 1
    
p_value = 1.0 - (np.mean(pval_list))/sample_size 
print("Posting volume p-value = ", p_value)

Posting volume p-value =  0.0


## 2) Rate of newcomers

In [37]:
from random import choices

treatment_coeff = -1.5854

control_coeffs = results['nu-coeff'].tolist()
# control_coeffs = post_volume_coeff_list

sample_size = 50
iteration = 0
limit = 10000

pval_list = []

while(iteration < limit):
    iteration_coeffs = choices(control_coeffs, k=sample_size)
    pval = 0
    for coeff in iteration_coeffs:
        if treatment_coeff < coeff:
            pval += 1
    pval_list.append(pval)
    iteration += 1
    
p_value = 1.0 - (np.mean(pval_list))/sample_size 
print("New users p-value = ", p_value)

New users p-value =  0.0


## 3) Rate of post removal

In [38]:
from random import choices

treatment_coeff = -0.0253

control_coeffs = results['rr-coeff'].dropna().tolist()
# control_coeffs = removal_rate_coeff_list

sample_size = 50
iteration = 0
limit = 10000

pval_list = []

while(iteration < limit):
    iteration_coeffs = choices(control_coeffs, k=sample_size)
    pval = 0
    for coeff in iteration_coeffs:
        if treatment_coeff < coeff:
            pval += 1
    pval_list.append(pval)
    iteration += 1
    
p_value = 1.0 - (np.mean(pval_list))/sample_size 
print("Removal rate p-value = ", p_value)

Removal rate p-value =  0.0


# Summary: Causal effect of Quarantining the r/TheRedPill compared to Reddit-wide effects

The effects observed within r/TRP was found to be "extreme" through bootstrapping tests, and we rejected the null hypothesis that the observed effects were caused due to a random, Reddit-wide event and not due to the quarantining of the subreddit.

### In other words, all effects observed within r/TRP were found to be caused due to the quarantining event, thereby establishing the causal link!